In [2]:
import pandas as pd
df = pd.read_csv('data.csv', index_col='date', parse_dates=True)

In [3]:
df.head()

,forecast
date,
2014-01-01,8016.0
2014-01-01,8016.0
2014-01-01,7767.0
2014-01-01,7767.0
2014-01-01,7408.0


In [4]:
df.index

DatetimeIndex(['2014-01-01', '2014-01-01', '2014-01-01', '2014-01-01',
               '2014-01-01', '2014-01-01', '2014-01-01', '2014-01-01',
               '2014-01-01', '2014-01-01',
               ...
               '2014-10-31', '2014-10-31', '2014-10-31', '2014-10-31',
               '2014-10-31', '2014-10-31', '2014-10-31', '2014-10-31',
               '2014-10-31', '2014-10-31'],
              dtype='datetime64[ns]', name='date', length=30300, freq=None)

In [5]:
df.resample(rule='W')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 30300 entries, 2014-01-01 to 2014-10-31
Data columns (total 1 columns):
forecast    29088 non-null float64
dtypes: float64(1)
memory usage: 473.4 KB


In [7]:
df['forecast'].fillna((df['forecast'].mean()), inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 30300 entries, 2014-01-01 to 2014-10-31
Data columns (total 1 columns):
forecast    30300 non-null float64
dtypes: float64(1)
memory usage: 473.4 KB


In [9]:
train_data = df.iloc[:24241] # Goes up to but not including 108
test_data = df.iloc[24241:]

In [10]:

train_data.shape

(24241, 1)

In [11]:
test_data.shape

(6059, 1)

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(train_data)


MinMaxScaler(copy=True, feature_range=(0, 1))

In [13]:
scaled_train=scaler.transform(train_data)
scaled_test=scaler.transform(test_data)

In [14]:
scaled_test.shape

(6059, 1)

In [15]:
import keras
from keras.preprocessing.sequence import TimeseriesGenerator

Using TensorFlow backend.


In [16]:
n_input=2
n_features=1
generator=TimeseriesGenerator(scaled_train,scaled_train,length=n_input,batch_size=1)

In [17]:
len(scaled_train)

24241

In [18]:
len(generator)

24239

In [19]:
X,y=generator[0]

In [20]:
X.shape


(1, 2, 1)

In [21]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [22]:
n_input=7
n_features=1
train_generator=TimeseriesGenerator(scaled_train,scaled_train,length=n_input,batch_size=1)

In [23]:
model = Sequential()
model.add(LSTM(256,activation='relu',input_shape=(n_input,n_features)))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse',metrics=['mae'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 264,449
Trainable params: 264,449
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(train_generator,epochs=5,steps_per_epoch=240)


Epoch 1/5
240/240 [==============================] - 8s 34ms/step - loss: 0.0118 - mae: 0.0732: 0s - loss: 0.0117 - mae:
Epoch 2/5
240/240 [==============================] - 6s 25ms/step - loss: 0.0037 - mae: 0.0458
Epoch 3/5
240/240 [==============================] - 6s 26ms/step - loss: 0.0034 - mae: 0.0453: 1s - loss: 0.
Epoch 4/5
240/240 [==============================] - 6s 26ms/step - loss: 0.0024 - mae: 0.0384: 2s - loss: 0.0 - ETA: 1s - los
Epoch 5/5
240/240 [==============================] - 7s 28ms/step - loss: 0.0035 - mae: 0.0445


In [26]:
model.history.history.keys()

dict_keys(['loss', 'mae'])

In [27]:
import matplotlib.pyplot as plt
plt.plot(range(len(model.history.history['mae'])),model.history.history['mae'])

In [28]:
first_eval_batch=scaled_test[1:8]

In [29]:
scaled_test[1:8]

array([[0.20330306],
       [0.20330306],
       [0.2087531 ],
       [0.2087531 ],
       [0.2360033 ],
       [0.2360033 ],
       [0.23616846]])

In [30]:
first_eval_batch

array([[0.20330306],
       [0.20330306],
       [0.2087531 ],
       [0.2087531 ],
       [0.2360033 ],
       [0.2360033 ],
       [0.23616846]])

In [31]:
first_eval_batch= first_eval_batch.reshape((1,n_input,n_features))

In [32]:
q=model.predict(first_eval_batch)

In [33]:
q

array([[0.24871115]], dtype=float32)

In [34]:
import numpy as np
first_eval_batch= scaled_train[-n_input:]
current_batch=first_eval_batch.reshape((1,n_input,n_features))

In [35]:
my_pred=model.predict(current_batch)

In [36]:
my_pred

array([[0.19583556]], dtype=float32)